In [55]:
import requests
import pandas as pd

from datetime import datetime, timedelta
import time
import json
import re

import sqlite3

In [61]:
def convert_date(date_str):
    year = int(date_str[:3]) + 1911
    month = int(date_str[4:6])
    day = int(date_str[7:9])
    return f"{year}-{month:02d}-{day:02d}"

def save_to_database(df, db_name="stocks.db"):
    conn = sqlite3.connect(db_name)
    df.to_sql("stocks", conn, if_exists="replace", index=False)
    conn.close()
# from 1 to 13
refs = {
    "第一款": 1,
    "第二款": 2,
    "第三款": 3,
    "第四款": 4,
    "第五款": 5,
    "第六款": 6,
    "第七款": 7,
    "第八款": 8,
    "第九款": 9,
    "第十款": 10,
    "第十一款": 11,
    "第十二款": 12,
    "第十三款": 13
}

In [73]:
datas=[]
with open("TSE_notice.json", "r", encoding="utf-8") as f:
    TSE_data = json.load(f)
    # datas.append(TSE_data)
with open("OTC_notice.json", "r", encoding="utf-8") as f:
    OTC_data = json.load(f)
    datas.append(OTC_data)

for data_source in datas:
    if data_source == TSE_data:
        data = pd.DataFrame(data_source["data"], columns=data_source["fields"])
        data['Source'] = 'TSE'
    elif data_source == OTC_data:
        data = pd.DataFrame(data_source['tables'][0]["data"], columns=data_source['tables'][0]["fields"])
        data.rename(columns={
            "累計": "累計次數",
            "公告日期": "日期"
        }, inplace=True)
        data['Source'] = 'OTC'


    data["證券代號"] = data["證券代號"].astype(str)
    data["累計次數"] = data["累計次數"].astype(object)
    data = data[data["證券代號"].str.len() == 4]
    data = data[data["證券代號"] > "1000"]

    if data_source == TSE_data:
        data["注意交易資訊"] = data["注意交易資訊"].apply(lambda x: [refs[item.split("﹝")[1]] for item in x.split("﹞")[:-1]])
    elif data_source == OTC_data:
        data["注意交易資訊"] = data["注意交易資訊"].apply(
            lambda x: [
                refs[m]
                for m in re.findall(r'[（(](第[一二三四五六七八九十百零]+款)[)）]', str(x))
                if m in refs
            ]
        )
        data = data.drop(columns=['link'])

    data["日期"] = data["日期"].apply(convert_date)
    # Convert the list in "注意交易資訊" to a JSON string before saving to the database
    data["注意交易資訊"] = data["注意交易資訊"].apply(lambda x: json.dumps(x, ensure_ascii=False))
    
    # Save DataFrame to SQLite database
    save_to_database(data)

In [74]:
df= pd.read_sql("SELECT * FROM stocks", sqlite3.connect("stocks.db"))
df

,編號,證券代號,證券名稱,累計次數,注意交易資訊,日期,收盤價,本益比,Source
0,1,3162,精確,22,"[3, 4]",2025-08-29,67.70,27.63,OTC
1,1,3163,波若威,22,[13],2025-08-29,248.00,94.30,OTC
2,1,3219,倚強科,22,"[1, 2, 2, 4]",2025-08-29,108.00,128.57,OTC
3,1,3297,杭特,22,[10],2025-08-29,66.10,472.14,OTC
4,1,3685,元創精密,22,"[1, 3]",2025-08-29,20.75,N/A,OTC
...,...,...,...,...,...,...,...,...,...
931,64,6147,頎邦,14,[12],2025-06-03,62.80,13.08,OTC
932,64,6148,驊宏資,14,[13],2025-06-03,63.00,43.15,OTC
933,64,6276,安鈦克,14,[13],2025-06-03,52.30,N/A,OTC
934,64,6593,台灣銘板,14,[1],2025-06-03,70.50,115.57,OTC
